In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lung_Cancer"
cohort = "GSE249568"

# Input paths
in_trait_dir = "../../input/GEO/Lung_Cancer"
in_cohort_dir = "../../input/GEO/Lung_Cancer/GSE249568"

# Output paths
out_data_file = "../../output/preprocess/Lung_Cancer/GSE249568.csv"
out_gene_data_file = "../../output/preprocess/Lung_Cancer/gene_data/GSE249568.csv"
out_clinical_data_file = "../../output/preprocess/Lung_Cancer/clinical_data/GSE249568.csv"
json_path = "../../output/preprocess/Lung_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression from paired biopsies from a patient with METex14 skiping non-small cell lung cancer before and after treatment with neoadjuvant tepotinib (42 days)"
!Series_summary	"Background: MET inhibitors have demonstrated efficacy in treating patients with non-small cell lung cancer (NSCLC) harboring METex14 skipping alterations. Advancements in spatial profiling technologies have unveiled the complex dynamics of the tumor microenvironment, a crucial factor in cancer progression and therapeutic response. Using spatial profiling, this study investigates the effects of the MET inhibitor tepotinib on the TME in a case of locally advanced NSCLC with a METex14 skipping alteration.  Methods: A patient with resectable stage IIIB NSCLC, unresponsive to neoadjuvant platinum-based doublet chemotherapy, received tepotinib following detection of a METex14 skipping alteration. Paired pre- and post-treatment biopsies were subjected to GeoMx Digital Spatial

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Let's first determine gene expression availability
# Based on the background information, this dataset appears to have gene expression data from biopsy samples
is_gene_available = True

# Now let's analyze the sample characteristics dictionary to find trait, age, and gender information
print("Sample characteristics dictionary:", {0: ['tissue: NSCLC']})

# 1. Trait (Lung Cancer) data:
# From the sample characteristics, all samples are marked as "tissue: NSCLC" (Non-Small Cell Lung Cancer)
# Since all samples have the same value (NSCLC), this is a constant feature
# According to our rules, constant features are not useful for associative studies
trait_row = None

# 2. Age data: 
# No age information is provided in the sample characteristics
age_row = None

# 3. Gender data:
# No gender information is provided in the sample characteristics
gender_row = None

# Define conversion functions (even though we won't use them in this case)
def convert_trait(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # In this case, all are NSCLC, but we would convert to binary if we had control samples
    if 'NSCLC' in value:
        return 1  # Has lung cancer
    else:
        return 0  # Control/normal

def convert_age(value):
    # Not needed for this dataset, but including for completeness
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)  # Age as a continuous variable
    except:
        return None

def convert_gender(value):
    # Not needed for this dataset, but including for completeness
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    else:
        return None

# Save metadata about dataset usability
# Since trait_row is None, is_trait_available is False
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since trait_row is None, we'll skip the clinical feature extraction step


Sample characteristics dictionary: {0: ['tissue: NSCLC']}


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Gene expression from paired biopsies from a patient with METex14 skiping non-small cell lung cancer before and after treatment with neoadjuvant tepotinib (42 days)"
Line 1: !Series_geo_accession	"GSE249568"
Line 2: !Series_status	"Public on Sep 27 2024"
Line 3: !Series_submission_date	"Dec 06 2023"
Line 4: !Series_last_update_date	"Sep 28 2024"
Line 5: !Series_summary	"Background: MET inhibitors have demonstrated efficacy in treating patients with non-small cell lung cancer (NSCLC) harboring METex14 skipping alterations. Advancements in spatial profiling technologies have unveiled the complex dynamics of the tumor microenvironment, a crucial factor in cancer progression and therapeutic response. Using spatial profiling, this study investigates the effects of the MET inhibitor tepotinib on the TME in a case of locally advanced NSCLC with a METex14 skipping alteration.  Methods: A patient with resectable stage IIIB NSCLC, unrespon

### Step 4: Gene Identifier Review

In [5]:
# Examine the gene IDs to determine if they are human gene symbols or other identifiers
# Looking at the first 20 gene IDs: A2M, ABCB1, ABCF1, ABL1, etc.
# These appear to be standard human gene symbols

# Based on my biomedical knowledge, these are proper human gene symbols
# A2M is Alpha-2-Macroglobulin
# ABCB1 is ATP Binding Cassette Subfamily B Member 1
# ABCF1 is ATP Binding Cassette Subfamily F Member 1
# ABL1 is ABL Proto-Oncogene 1, Non-Receptor Tyrosine Kinase
# ACTA2 is Actin Alpha 2, Smooth Muscle
# ACTB is Actin Beta

# These are standard HGNC (HUGO Gene Nomenclature Committee) approved symbols
# No mapping to gene symbols is required as the data already uses them

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols using NCBI Gene database
print("\nNormalizing gene symbols using NCBI Gene database...")
try:
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(gene_data_normalized)} unique genes remain")
    gene_data_cleaned = gene_data_normalized
    
    # Save the normalized gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_cleaned.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene symbol normalization: {str(e)}")
    print("Falling back to original gene data")
    gene_data_cleaned = gene_data.copy()

# 2. We need to recreate the clinical data from the original matrix file
print("\nExtracting clinical data directly from the matrix file...")
# Extract clinical data from the matrix file again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Process clinical data
print("\nProcessing clinical data...")
# Create clinical features dataframe 
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Extracted clinical data with shape: {selected_clinical_df.shape}")
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")
    
    is_trait_available = True
else:
    selected_clinical_df = pd.DataFrame()
    is_trait_available = False
    print("No trait data available in clinical information.")

# 3. Link clinical and genetic data
if is_trait_available and is_gene_available:
    print("\nLinking clinical and genetic data...")
    try:
        # Ensure the sample IDs match between clinical and genetic data
        common_samples = list(set(selected_clinical_df.columns).intersection(set(gene_data_cleaned.columns)))
        
        if len(common_samples) == 0:
            print("Warning: No common samples between clinical and genetic data")
            linked_data = pd.DataFrame()
            is_biased = True
        else:
            print(f"Found {len(common_samples)} common samples between clinical and genetic data")
            
            # Filter data to include only common samples
            clinical_subset = selected_clinical_df[common_samples]
            genetic_subset = gene_data_cleaned[common_samples]
            
            # Link the data
            linked_data = pd.concat([clinical_subset, genetic_subset], axis=0).T
            print(f"Linked data shape: {linked_data.shape}")
            
            # 4. Handle missing values
            print("\nHandling missing values in linked data...")
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Data shape after handling missing values: {linked_data.shape}")
            
            # 5. Determine if trait and demographic features are severely biased
            print("\nEvaluating trait and demographic feature bias...")
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    except Exception as e:
        print(f"Error during data linking: {str(e)}")
        linked_data = pd.DataFrame()
        is_biased = True
else:
    print("\nCannot create linked data: missing clinical or gene data")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final validation and saving
note = "This dataset contains gene expression data from astrocytoma cell lines with modified GFAP isoform expression. The trait represents different experimental conditions related to the GFAPδ/GFAPα ratio."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if len(linked_data) > 0 else True,
    df=linked_data,
    note=note
)

# Save the linked data if it's usable
if is_usable and len(linked_data) > 0:
    print(f"\nSaving linked data to {out_data_file}")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved successfully!")
else:
    print(f"\nDataset not usable for {trait} association studies due to bias or quality issues.")


Normalizing gene symbols using NCBI Gene database...
After normalization: 1798 unique genes remain


Normalized gene expression data saved to ../../output/preprocess/Lung_Cancer/gene_data/GSE249568.csv

Extracting clinical data directly from the matrix file...

Processing clinical data...
No trait data available in clinical information.

Cannot create linked data: missing clinical or gene data
Abnormality detected in the cohort: GSE249568. Preprocessing failed.

Dataset not usable for Lung_Cancer association studies due to bias or quality issues.
